<a href="https://colab.research.google.com/github/lachlandeer/ruokay-tweets-analysis/blob/main/ruokay_tweets_selector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd


In [3]:
from google.colab import auth
auth.authenticate_user()

## Get Data from Googel Storage

In [4]:
project_id = 'ruokay-tweets'


In [5]:
!gcloud config set project {project_id}
!gsutil ls

Updated property [core/project].
gs://ruokay/


In [6]:
!mkdir data


In [10]:
from google.cloud import storage
import json
import re

In [11]:
# Initialize Google Cloud Storage client
client = storage.Client()
bucket_name = 'ruokay'
bucket = client.get_bucket(bucket_name)

In [12]:
# Pattern to match "data_SOMETHING.json"
pattern = r"data_\d+\.json"

# List blobs (files) in the broader directory
blobs = bucket.list_blobs(prefix='data/json/')

In [25]:
# Initialize list to store DataFrames
data_frames = []

# Loop through all blobs in the directory, filter, and load matching JSON files
for blob in blobs:
    if re.search(pattern, blob.name):  # Check if the file name matches the pattern
        data = blob.download_as_text()  # Download JSON content as text
        json_data = json.loads(data)    # Parse JSON
        df = pd.DataFrame(json_data)    # Convert JSON data to DataFrame
        data_frames.append(df)          # Append DataFrame to list

# Combine all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Display the combined DataFrame
combined_df.head()

ValueError: ('Iterator has already started', <google.api_core.page_iterator.HTTPIterator object at 0x794562178a60>)

In [26]:
combined_df.head()

,public_metrics,created_at,text,id,source,lang,author_id,possibly_sensitive,entities,conversation_id,referenced_tweets,in_reply_to_user_id,geo,attachments,withheld,edit_history_tweet_ids
0,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2009-07-31T13:34:57.000Z,Off work today. Thinking about @BeyondBlue bre...,2950984512,Twitter Web Client,en,17145537,False,"{'mentions': [{'start': 31, 'end': 42, 'userna...",2950984512,{},NaN,NaN,NaN,NaN,NaN
1,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2009-07-31T10:13:03.000Z,G'Day @Ike_Pono @ukresident @Hispanic_Views @C...,2948886230,Twitter Web Client,en,29625623,False,"{'mentions': [{'start': 6, 'end': 15, 'usernam...",2948886230,{},NaN,NaN,NaN,NaN,NaN
2,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2009-07-31T05:49:31.000Z,NEW JOB: Program Managers Consumers and Carers...,2946359039,Twitter Web Client,en,58663701,False,"{'mentions': {}, 'annotations': {}, 'hashtags'...",2946359039,{},NaN,NaN,NaN,NaN,NaN
3,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2009-07-30T23:47:34.000Z,"BeyondBlue, Black Dog Institute, Brain and Min...",2940665877,Twitter Web Client,en,30584764,False,"{'mentions': [{'start': 120, 'end': 132, 'user...",2940665877,{},NaN,NaN,NaN,NaN,NaN
4,"{'retweet_count': 0, 'reply_count': 0, 'like_c...",2009-07-29T15:54:24.000Z,The Red Jumpsuit Apparatus – Face Down... Sho...,2911641970,Twitter Web Client,en,40431476,False,"{'mentions': [{'start': 64, 'end': 75, 'userna...",2911641970,{},NaN,NaN,NaN,NaN,NaN


In [27]:
# get the columns we need
df = combined_df[['id', 'created_at', 'text', 'lang']]


In [17]:
df.shape

(1306625, 4)

In [18]:
# Get memory usage in bytes
memory_usage_bytes = df.memory_usage(deep=True).sum()
print(f"Memory usage: {memory_usage_bytes / (1024 ** 2):.2f} MB")

Memory usage: 674.96 MB


## Lets select tweets near RUOK Day each year

In [28]:
# First lets format the time stamp and convert to sydney time
# Step 1: Convert 'created_at' to datetime format
df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')

# Step 2: Check if 'created_at' is now a datetime dtype
print(df['created_at'].dtype)  # Should display datetime64[ns, UTC] after the steps below

# Step 3: Localize 'created_at' to UTC if not already timezone-aware
if df['created_at'].dt.tz is None:
    df['created_at'] = df['created_at'].dt.tz_localize('UTC')

# Step 4: Convert 'created_at' to Sydney time
df['created_at'] = df['created_at'].dt.tz_convert('Australia/Sydney')


datetime64[ns]


<ipython-input-28-8c4535ca94d2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = pd.to_datetime(df['created_at'], errors='coerce', format='%Y-%m-%dT%H:%M:%S.%fZ')
<ipython-input-28-8c4535ca94d2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['created_at'] = df['created_at'].dt.tz_localize('UTC')
<ipython-input-28-8c4535ca94d2>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [29]:
df.head()

,id,created_at,text,lang
0,2950984512,2009-07-31 23:34:57+10:00,Off work today. Thinking about @BeyondBlue bre...,en
1,2948886230,2009-07-31 20:13:03+10:00,G'Day @Ike_Pono @ukresident @Hispanic_Views @C...,en
2,2946359039,2009-07-31 15:49:31+10:00,NEW JOB: Program Managers Consumers and Carers...,en
3,2940665877,2009-07-31 09:47:34+10:00,"BeyondBlue, Black Dog Institute, Brain and Min...",en
4,2911641970,2009-07-30 01:54:24+10:00,The Red Jumpsuit Apparatus – Face Down... Sho...,en


In [30]:
# Extract the year from the 'created_at' column in the Twitter dataset
df['year'] = df['created_at'].dt.year

<ipython-input-30-d7d86bd2819f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['year'] = df['created_at'].dt.year


In [31]:
# Function to find the second Thursday of September in a given year
def second_thursday_of_september(year):
    date = datetime(year, 9, 1)
    first_thursday = date + timedelta(days=(3 - date.weekday() + 7) % 7)
    second_thursday = first_thursday + timedelta(days=7)
    return second_thursday

# Create a DataFrame with second Thursdays for each year
second_thursdays = pd.DataFrame({
    'year': range(2009, 2020),
    'second_thursday': [second_thursday_of_september(year) for year in range(2009, 2020)]
})

In [33]:
# Merge the Twitter dataset with the second Thursdays DataFrame on 'year'
merged_data = pd.merge(df, second_thursdays, on='year', how='left')

In [35]:
merged_data.head()

,id,created_at,text,lang,year,second_thursday
0,2950984512,2009-07-31 23:34:57+10:00,Off work today. Thinking about @BeyondBlue bre...,en,2009,2009-09-10
1,2948886230,2009-07-31 20:13:03+10:00,G'Day @Ike_Pono @ukresident @Hispanic_Views @C...,en,2009,2009-09-10
2,2946359039,2009-07-31 15:49:31+10:00,NEW JOB: Program Managers Consumers and Carers...,en,2009,2009-09-10
3,2940665877,2009-07-31 09:47:34+10:00,"BeyondBlue, Black Dog Institute, Brain and Min...",en,2009,2009-09-10
4,2911641970,2009-07-30 01:54:24+10:00,The Red Jumpsuit Apparatus – Face Down... Sho...,en,2009,2009-09-10


In [39]:
# Localize 'second_thursday' to the same timezone as 'created_at'
merged_data['second_thursday'] = merged_data['second_thursday'].dt.tz_localize('Australia/Sydney')
# Calculate the difference in days realtive to RUOK day
merged_data['days_from_thursday'] = (merged_data['created_at'] - merged_data['second_thursday']).dt.days


In [42]:
merged_data.head()

,id,created_at,text,lang,year,second_thursday,days_from_thursday
0,2950984512,2009-07-31 23:34:57+10:00,Off work today. Thinking about @BeyondBlue bre...,en,2009,2009-09-10 00:00:00+10:00,-41.0
1,2948886230,2009-07-31 20:13:03+10:00,G'Day @Ike_Pono @ukresident @Hispanic_Views @C...,en,2009,2009-09-10 00:00:00+10:00,-41.0
2,2946359039,2009-07-31 15:49:31+10:00,NEW JOB: Program Managers Consumers and Carers...,en,2009,2009-09-10 00:00:00+10:00,-41.0
3,2940665877,2009-07-31 09:47:34+10:00,"BeyondBlue, Black Dog Institute, Brain and Min...",en,2009,2009-09-10 00:00:00+10:00,-41.0
4,2911641970,2009-07-30 01:54:24+10:00,The Red Jumpsuit Apparatus – Face Down... Sho...,en,2009,2009-09-10 00:00:00+10:00,-42.0


In [45]:
# Filter tweets within ±7 days of 'second_thursday'
date_range_days = 28
filtered_data = merged_data[
    (merged_data['days_from_thursday'].abs() <= date_range_days) &
    (merged_data['year'].between(2011, 2019))
    ]


In [46]:
filtered_data.head()

,id,created_at,text,lang,year,second_thursday,days_from_thursday
15862,107321434191298561,2011-08-27 15:19:58+10:00,RT @Abbeyrescuedox: RT @HumanityRoad: In a dis...,en,2011,2011-09-08 00:00:00+10:00,-12.0
15863,107318405278859265,2011-08-27 15:07:56+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,en,2011,2011-09-08 00:00:00+10:00,-12.0
15864,107318045436936193,2011-08-27 15:06:30+10:00,RT @NRL_Bulldogs: It's go time! Game day & @AN...,en,2011,2011-09-08 00:00:00+10:00,-12.0
15865,107316121253855232,2011-08-27 14:58:51+10:00,It's go time! Game day & @ANZStadium looks a p...,en,2011,2011-09-08 00:00:00+10:00,-12.0
15866,107299839867494400,2011-08-27 13:54:10+10:00,@jpdl_us there's nothing stupid about ruok imo...,en,2011,2011-09-08 00:00:00+10:00,-12.0


In [48]:
filtered_data.shape

(414886, 7)

## Save to GCS

In [49]:
# Step 1: Save the filtered DataFrame as a temporary CSV file
filtered_data.to_csv('/tmp/ruokay_tweets_28_days_2011_2019.csv', index=False)

In [50]:
# Step 2: Upload the CSV file to Google Cloud Storage
destination_blob_name = 'out_data/ruokay_tweets_28_days_2011_2019.csv'  # Specify the folder path
# Get the bucket and upload the file
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(destination_blob_name)
blob.upload_from_filename('/tmp/ruokay_tweets_28_days_2011_2019.csv')

print(f"File uploaded to {bucket_name}/{destination_blob_name}")

File uploaded to ruokay/out_data/ruokay_tweets_28_days_2011_2019.csv
